In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/gurgaon_properties_post_feature_selection (1).csv')

In [ ]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,36.0,3,2,2.0,1.0,850.0,0,0,0,1.0,1.0,0.82
1,0.0,95.0,2,2,2.0,1.0,1226.0,1,0,0,1.0,2.0,0.95
2,0.0,103.0,2,2,1.0,1.0,1000.0,0,0,0,1.0,0.0,0.32
3,0.0,99.0,3,4,4.0,3.0,1615.0,1,0,1,0.0,2.0,1.60
4,0.0,5.0,2,2,1.0,3.0,582.0,0,1,0,0.0,2.0,0.48


In [ ]:
#Now we are going to apply linear regression i will perform one hot encoding as categorical cols are ordinally encoded and that could lead to wrong predictions
#then i will scale the values
#then is will perform log transformation for the skewed data
X=df.drop(columns=['price'])
y=df['price']

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [ ]:
columns_to_encode=['sector','balcony','agePossession','furnishing_type','floor_category','luxury_category']

In [ ]:
y_transformed=np.log1p(y)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type','bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ],
    remainder='passthrough'
)

In [ ]:
pipeline=Pipeline([
    ('preprocessor', preprocessor),
    ('regressor',SVR(kernel='rbf'))
])

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [ ]:
scores.mean()

np.float64(0.8838614878551869)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [ ]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'floor_category',
                                                   'luxury_category'])])),
                ('regressor', SVR())])

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
y_pred = np.expm1(y_pred)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5428402722157795

In [ ]:
df=pd.read_csv('/content/without_encoding_feature_selection.csv')

In [ ]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3,2,2,New Property,850.0,0,0,0,Low,Low Rise
1,flat,sector 89,0.95,2,2,2,New Property,1226.0,1,0,0,Low,Mid Rise
2,flat,sohna road,0.32,2,2,1,New Property,1000.0,0,0,0,Low,High Rise
3,flat,sector 92,1.60,3,4,3+,Relatively New,1615.0,1,0,1,High,Mid Rise
4,flat,sector 102,0.48,2,2,1,Relatively New,582.0,0,1,0,High,Mid Rise


In [ ]:
df['furnishing_type'].value_counts()

,count
furnishing_type,
0,2349
1,1018
2,187


In [ ]:
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished',1.0:'semifurnished',2.0:'furnished'})

In [ ]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3,2,2,New Property,850.0,0,0,unfurnished,Low,Low Rise
1,flat,sector 89,0.95,2,2,2,New Property,1226.0,1,0,unfurnished,Low,Mid Rise
2,flat,sohna road,0.32,2,2,1,New Property,1000.0,0,0,unfurnished,Low,High Rise
3,flat,sector 92,1.60,3,4,3+,Relatively New,1615.0,1,0,semifurnished,High,Mid Rise
4,flat,sector 102,0.48,2,2,1,Relatively New,582.0,0,1,unfurnished,High,Mid Rise


In [ ]:
X = df.drop(columns=['price'])
y = df['price']

In [ ]:
y_transformed = np.log1p(y)

In [ ]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1), columns_to_encode)
    ],
    remainder='passthrough'
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
def scorer(model_name,model):
  output=[]
  output.append(model_name)
  pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', model)
])

  kfold = KFold(n_splits=10, shuffle=True, random_state=42)
  scores = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='r2')

  output.append(scores.mean())


  pipeline.fit(X_train,y_train)

  y_pred = pipeline.predict(X_test)

  y_pred = np.expm1(y_pred)

  output.append(mean_absolute_error(np.expm1(y_test),y_pred))

  return output


In [ ]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'xgboost':XGBRegressor()
}

In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [ ]:
model_output

[['linear_reg', np.float64(0.7334247718919744), 0.9473651486336957],
 ['svr', np.float64(0.753922785307853), 0.8476956943969968],
 ['ridge', np.float64(0.7334362777802037), 0.9473190325425672],
 ['LASSO', np.float64(0.059050591664503514), 1.528905986892753],
 ['decision tree', np.float64(0.7539211788123122), 0.7274285986900951],
 ['random forest', np.float64(0.869892912329371), 0.5232316652023123],
 ['extra trees', np.float64(0.8533039276325625), 0.5509222917797844],
 ['gradient boosting', np.float64(0.8694935766870422), 0.5765724933586264],
 ['adaboost', np.float64(0.7564017803896551), 0.8383924170745537],
 ['xgboost', np.float64(0.8836281374209698), 0.5078911675741233]]

In [ ]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [ ]:
model_df.sort_values(['mae'])

,name,r2,mae
9,xgboost,0.883628,0.507891
5,random forest,0.869893,0.523232
6,extra trees,0.853304,0.550922
7,gradient boosting,0.869494,0.576572
4,decision tree,0.753921,0.727429
8,adaboost,0.756402,0.838392
1,svr,0.753923,0.847696
2,ridge,0.733436,0.947319
0,linear_reg,0.733425,0.947365
3,LASSO,0.059051,1.528906


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False, handle_unknown='ignore'),['sector','agePossession','furnishing_type'])
    ],
    remainder='passthrough'
)

In [ ]:
def scorer(model_name, model):

    output = []

    output.append(model_name)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline,X_train, y_train, cv=kfold, scoring='r2')

    output.append(scores.mean())


    pipeline.fit(X_train,y_train)

    y_pred = pipeline.predict(X_test)

    y_pred = np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))

    return output


In [ ]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'xgboost':XGBRegressor()
}

In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categ

In [ ]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [ ]:
model_df.sort_values(['mae'])

,name,r2,mae
6,extra trees,0.883109,0.469696
9,xgboost,0.889329,0.492317
5,random forest,0.877292,0.502849
7,gradient boosting,0.868480,0.574387
0,linear_reg,0.847360,0.648482
2,ridge,0.847392,0.652102
4,decision tree,0.782671,0.688474
1,svr,0.757629,0.834487
8,adaboost,0.755412,0.842228
3,LASSO,0.059051,1.528906


In [ ]:
import category_encoders as ce

columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False,handle_unknown='ignore'),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ],
    remainder='passthrough'
)

In [ ]:
def scorer(model_name, model):

    output = []

    output.append(model_name)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='r2')

    output.append(scores.mean())

    pipeline.fit(X_train,y_train)

    y_pred = pipeline.predict(X_test)

    y_pred = np.expm1(y_pred)

    output.append(mean_absolute_error(np.expm1(y_test),y_pred))

    return output


In [ ]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [ ]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [ ]:
model_df.sort_values(['mae'])

,name,r2,mae
9,xgboost,0.889179,0.441126
6,extra trees,0.891150,0.460288
5,random forest,0.890488,0.466434
7,gradient boosting,0.882949,0.510408
4,decision tree,0.809789,0.585752
8,adaboost,0.816744,0.678543
0,linear_reg,0.821005,0.711703
2,ridge,0.821036,0.712251
1,svr,0.766835,0.818564
3,LASSO,0.059051,1.528906


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rf_params = {
    'regressor__n_estimators': [100, 300, 500, 800],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__max_samples': [0.25, 0.5, 0.75, 1.0],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__max_features': ['sqrt', 'log2', None]
}

et_params = rf_params.copy()

xgb_params = {
    'regressor__n_estimators': [100, 500, 1000],
    'regressor__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'regressor__max_depth': [3, 6, 9],
    'regressor__subsample': [0.7, 0.8, 0.9],
    'regressor__colsample_bytree': [0.7, 0.8, 0.9],
    'regressor__gamma': [0, 0.1, 0.2]
}
models = [
    ('RandomForest', RandomForestRegressor(random_state=42,bootstrap=True), rf_params),
    ('ExtraTrees', ExtraTreesRegressor(random_state=42,bootstrap=True), et_params),
    ('XGBoost', XGBRegressor(objective='reg:squarederror', random_state=42), xgb_params)
]


In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
best_pipelines = {}

In [ ]:
for name, model, params in models:
    print(f"--- Starting tuning for {name} ---")
    current_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    rs = RandomizedSearchCV(
        estimator=current_pipeline,
        param_distributions=params,
        n_iter=15,
        cv=kfold,
        scoring='r2',
        verbose=4,
        random_state=42,
        n_jobs=-1
    )

    rs.fit(X_train, y_train)

    best_pipelines[name] = rs.best_estimator_
    print(f"Best R2 for {name}: {rs.best_score_}\n")

--- Starting tuning for RandomForest ---
Fitting 10 folds for each of 15 candidates, totalling 150 fits
Best R2 for RandomForest: 0.8922826942802324

--- Starting tuning for ExtraTrees ---
Fitting 10 folds for each of 15 candidates, totalling 150 fits
Best R2 for ExtraTrees: 0.8860586745095294

--- Starting tuning for XGBoost ---
Fitting 10 folds for each of 15 candidates, totalling 150 fits
Best R2 for XGBoost: 0.8974584218853014



In [ ]:
rs.best_params_

{'regressor__subsample': 0.7,
 'regressor__n_estimators': 500,
 'regressor__max_depth': 6,
 'regressor__learning_rate': 0.1,
 'regressor__gamma': 0,
 'regressor__colsample_bytree': 0.9}

In [ ]:
from sklearn.metrics import r2_score
y_pred_log = rs.best_estimator_.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_test_actual = np.expm1(y_test)
final_r2 = r2_score(y_test_actual, y_pred)
final_mae = mean_absolute_error(y_test_actual, y_pred)

print(f"Final Test R2: {final_r2:.4f}")
print(f"Final Test MAE: {final_mae:.2f}")

Final Test R2: 0.8426
Final Test MAE: 0.46


In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(objective='reg:squarederror', random_state=42))
])

In [ ]:
rs = RandomizedSearchCV(estimator=pipeline, param_distributions=xgb_params,n_iter=15,cv=kfold,scoring='r2',verbose=4, random_state=42,n_jobs=-1)

In [ ]:
rs.fit(X, y_transformed)

Fitting 10 folds for each of 15 candidates, totalling 150 fits


RandomizedSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('num',
                                                                               StandardScaler(),
                                                                               ['bedRoom',
                                                                                'bathroom',
                                                                                'built_up_area',
                                                                                'servant '
                                                                                'room',
                                                                                'store '
                                                                                'room']),
                                                                              ('cat',
                                                                               OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                              unknown_value=-1),
                                                                               ['property_ty...
                                                           n_jobs=None,
                                                           num_parallel_tree=None, ...))]),
                   n_iter=15, n_jobs=-1,
                   param_distributions={'regressor__colsample_bytree': [0.7,
                                                                        0.8,
                                                                        0.9],
                                        'regressor__gamma': [0, 0.1, 0.2],
                                        'regressor__learning_rate': [0.01, 0.05,
                                                                     0.1, 0.2],
                                        'regressor__max_depth': [3, 6, 9],
                                        'regressor__n_estimators': [100, 500,
                                                                    1000],
                                        'regressor__subsample': [0.7, 0.8,
                                                                 0.9]},
                   random_state=42, scoring='r2', verbose=4)

In [ ]:
rs.best_score_

np.float64(0.9073763466157285)

Exporting the pipleine and the data

In [ ]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False,handle_unknown='ignore'),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ],
    remainder='passthrough'
)

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(objective='reg:squarederror', random_state=42))
])

In [ ]:
final_model = rs.best_estimator_

In [ ]:
import pickle
with open('xgboost_pipeline.pkl','wb') as file:
  pickle.dump(final_model,file)

In [ ]:
with open('df.pkl','wb') as file:
  pickle.dump(X,file)

In [ ]:
X.columns

Index(['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')

In [ ]:
X.iloc[0].values

array(['flat', 'sector 36', np.int64(3), np.int64(2), '2', 'New Property',
       np.float64(850.0), np.int64(0), np.int64(0), 'unfurnished', 'Low',
       'Low Rise'], dtype=object)

In [ ]:
data = [['house', 'sector 102', 4, 3, '3+', 'New Property', 2750, 0, 0, 'unfurnished', 'Low', 'Low Rise']]
columns = ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category']
one_df = pd.DataFrame(data, columns=columns)

one_df

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,house,sector 102,4,3,3+,New Property,2750,0,0,unfurnished,Low,Low Rise


In [ ]:
with open('xgboost_pipeline.pkl', 'rb') as file:
    best_pipeline = pickle.load(file)
prediction_log = best_pipeline.predict(one_df)
final_price = np.expm1(prediction_log)
print(f"Predicted Price: {final_price[0]:.2f}")

Predicted Price: 2.39


In [ ]:
with open('df.pkl', 'rb') as file:
    df = pickle.load(file)
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,3,2,2,New Property,850.0,0,0,unfurnished,Low,Low Rise
1,flat,sector 89,2,2,2,New Property,1226.0,1,0,unfurnished,Low,Mid Rise
2,flat,sohna road,2,2,1,New Property,1000.0,0,0,unfurnished,Low,High Rise
3,flat,sector 92,3,4,3+,Relatively New,1615.0,1,0,semifurnished,High,Mid Rise
4,flat,sector 102,2,2,1,Relatively New,582.0,0,1,unfurnished,High,Mid Rise
